# Predict Genclasses

In [1]:
def get_entrez_id(hugo_df):
    
    """hugo_df - dataframe with Hugo_Symbol column"""
    
    import rpy2.robjects as ro
    from rpy2.robjects import pandas2ri
    
    pandas2ri.activate()

    run_get_entrez_id = ro.r(
        
        """
        library(biomaRt)

         function(hugo_df) {
            print('biomaRt is runnig and it can easily fail due to random server error, so be patient')
            
            mart <- useMart(biomart = "ensembl", dataset = "hsapiens_gene_ensembl")
            
            tmp_ids <- getBM(attributes = c("hgnc_symbol",  'entrezgene_id'),
                     filters = "hgnc_symbol",
                     values = hugo_df$Hugo_Symbol, 
                     mart = mart)
            
            tmp_ids <- as.data.frame(tmp_ids)
            
            return(tmp_ids)
        }
        """)

    get_entrez_id_results = run_get_entrez_id(hugo_df)


    return get_entrez_id_results

In [2]:
def hugo_to_entrez_map(hugo_df):
    """hugo_df - dataframe with Hugo_symbol column
       Returns dataframe with all original columns + ENTREZ_ID column"""
    
    import numpy as np
    
    hugo_to_entrez_dict = np.load('/uftp/projects/GenClass_Prediction/hugo_to_entrez_dict.npy', allow_pickle=True).item()
    
    hugo_df['Hugo_Symbol'] = hugo_df['Hugo_Symbol'].map(hugo_to_entrez_dict)
    
    hugo_df.dropna(inplace=True)
    
    hugo_df.rename(columns={'Hugo_Symbol': 'ENTREZ_ID'}, inplace=True)
    
    hugo_df.ENTREZ_ID = hugo_df.ENTREZ_ID.astype(int)
            
    return hugo_df

In [1]:
def process_mutation_file(maf):
    
    '''Maf file should be in format of S3Cohort'''
    
    
    print('Generating Mutation file')
    
    # Get entrez_id for genes coded with hugo_symbol
    
    maf = maf.astype(str)
    
    maf = hugo_to_entrez_map(maf)
    
    # Processing
    
    if 'Start_Position' in maf.columns:
        maf = maf[['Tumor_Sample_Barcode', 'ENTREZ_ID', 
                     'Variant_Classification', 'Start_Position']]
    else:
        maf = maf[['Tumor_Sample_Barcode', 'ENTREZ_ID', 
                     'Variant_Classification']]
        maf['Start_Position'] = -1
    
    rename_dict = {
        'Tumor_Sample_Barcode': 'Sample',
        'Variant_Classification': 'Type',
        'Start_Position': 'Location'
    }
    
    maf.rename(columns = rename_dict, inplace=True)
    
    # Reannotation of mutations
    # a. TRUNC: indicating a nonsense mutation in the coding region of the gene.
    # b. MUTATION: indicating a missense or frameshift mutation in the coding region of the gene.
    # c. Synon: indicating a mutation in the 5’UTR of the gene or a synonymous mutation in the
    # coding region within 4kb of the transcription start site.
    # d. L265P: indicating a L265P mutation of the MYD88 gene. (see below)
    
    reannotation = {
        'Missense_Mutation': 'MUTATION',
        'In_Frame_Ins': 'MUTATION',
        'In_Frame_Del': 'MUTATION',
        'Frame_Shift_Ins': 'TRUNC',
        'Nonsense_Mutation': 'TRUNC',
        'Frame_Shift_Del': 'TRUNC', 
        'Nonstop_Mutation': 'TRUNC',
        'Translation_Start_Site': 'TRUNC',
        'Splice_Site': 'TRUNC', 
        'Synonymous': 'Synon',
        'Silent': 'Synon',
        "3'UTR": 'Synon', 
        'UTR5': 'Synon', 
        'UTR3': 'Synon', 
        '3’ UTR': 'Synon', 
        "5'UTR": 'Synon', 
        "3'Flank": 'NA', 
        "5'Flank": 'NA', 
        'Splice_Region': 'NA',
        'IGR': 'NA', 
        'RNA': 'NA',
    }
    
    maf.Type = maf.Type.map(reannotation)
    
    maf.dropna(inplace=True)
    
    maf.sort_values('Sample', inplace=True)
    
    print('Mutation file done')
    
    
    return maf

In [4]:
def get_mutation_genlist(maf_processed):
    
    
    unique_maf_genes = pd.DataFrame(maf_processed.ENTREZ_ID.unique())
    unique_maf_genes.rename(columns={0: 'ENTREZ_ID'}, inplace=True)
    unique_maf_genes.sort_values('ENTREZ_ID', inplace=True)
    
    return unique_maf_genes

In [5]:
def get_segments_arm(segments):
    
    segments_arm = pd.DataFrame(gen_arms_nt(segments)).T
    
    segments_arm.fillna(0, inplace=True)
    
    segments_arm = segments_arm.astype(int)
    
    return segments_arm


def gen_arms_nt(segments):
    
    from tqdm import tqdm
    
    import warnings
    warnings.filterwarnings("ignore")
    
    from bioreactor.cna import process_segments, arm_ploidy_by_mode, normalized_total_cna, calc_ploidy_by_segments
    from gefest.utils import check_array_for_single_value
    
    segments['ploidy'] = calc_ploidy_by_segments(segments)
    
    for sample in tqdm(segments['Sample'].unique(), desc="{Generating segments_arm}"):
            
            sample_segments = segments[segments['Sample'] == sample][[
                'chrom',
                'start',
                'end',
                'n_mark',
                'minor',
                'total',
                'n_h_mark',
                'c_fraction',
                'ploidy',
                'Sample']]

            segments_processed = process_segments(sample_segments)

            arm_ploidy = arm_ploidy_by_mode(segments_processed)
            index = arm_ploidy['chrom'].astype(str).str.cat(arm_ploidy['arm'])
            index.name = 'arm'
            arm_ploidy = arm_ploidy.set_index(index)
            arm_ploidy_nt = normalized_total_cna(
                arm_ploidy.arm_total,
                check_array_for_single_value(segments['ploidy']))
            arm_ploidy_nt.name = sample


            yield arm_ploidy_nt.reindex([
                '1p', '1q', '2p', '2q', '3p', '3q', '4p', '4q', '5p', '5q', '6p', '6q', '7p',
                '7q', '8p', '8q', '9p', '9q', '10p', '10q', '11p', '11q', '12p', '12q', '13p',
                '13q', '14p', '14q', '15p', '15q', '16p', '16q', '17p', '17q', '18p', '18q',
                '19p', '19q', '20p', '20q', '21p', '21q', '22p', '22q', '23p', '23q'
            ])

In [6]:
def get_arm_file(segments_arm):
    
    from tqdm import tqdm
    
    """Input - arm segments. Returns a dataframe with 3 columns: Sample, Arm, Type, 
       where Type - cna event for specified arm"""
    
    cna_dict = {
        '2': 'AMP',
        '1': 'GAIN',
        '-1': 'HETLOSS',
        '-2': 'HOMDEL',
        'Gain': 'GAIN',
        'ShallowDel': 'HETLOSS',
        'Amplification': 'AMP',
        'Deletion': 'HOMDEL'
    }

    arm_df = {}

    for i in tqdm(segments_arm.columns, desc="{Generating arm_file}"):
        arm_df[i] = segments_arm[i][segments_arm[i] != 0]
    
    arm_df = pd.concat(arm_df)
    arm_df = arm_df.reset_index()
    arm_df.columns = ['Sample', 'Arm', 'Type']
    arm_df = arm_df.astype(str)
    arm_df.Type = arm_df.Type.map(cna_dict)
    arm_df.dropna(inplace=True)
    
    return(arm_df)

In [7]:
def get_cna_gene_file(segments_gene):
    
    from tqdm import tqdm
    
    """Input - gene segments. Genes - rows, Samples - columns. 
       Returns a dataframe with 4 columns: Sample, Hugo_Symbol, Type, Location 
       where Type - cna event for specified gene"""

    
    cna_dict = {
        '2': 'AMP',
        '1': 'GAIN',
        '-1': 'HETLOSS',
        '-2': 'HOMDEL',
        'Gain': 'GAIN',
        'ShallowDel': 'HETLOSS',
        'Amplification': 'AMP',
        'Deletion': 'HOMDEL'
    }
    
    gene_df = {}

    for i in tqdm(segments_gene.columns, desc="{Generating cna_gene_file}"):
        gene_df[i] = segments_gene[i][segments_gene[i] != 0]
    
    gene_df = pd.concat(gene_df)
    gene_df = gene_df.reset_index()
    gene_df.columns = ['Sample', 'Hugo_Symbol', 'Type']
    gene_df = gene_df.astype(str)
    gene_df.Type = gene_df.Type.map(cna_dict)
    gene_df.dropna(inplace=True)
    
    # Obtain entrez_id
    
    gene_df = hugo_to_entrez_map(gene_df)
    
    gene_df = gene_df[['Sample', 'ENTREZ_ID', 'Type']]
    
    gene_df['Location'] = -1
    
    return(gene_df)

In [8]:
def get_unique_cna_genes(cna_gene_file):
    
    """Input - cna gene file obtained with get_cna_gene_file. Returns dataframe with 1 column - ENTREZ_ID"""
    
    unique_cna_genes = pd.DataFrame(cna_gene_file.ENTREZ_ID.unique())
    unique_cna_genes.rename(columns={0: 'ENTREZ_ID'}, inplace=True)
    unique_cna_genes.sort_values('ENTREZ_ID', inplace=True)
    
    return unique_cna_genes

In [9]:
def sample_annotation(maf, cna_gene_file = None, fusion_data = None):
    
    from tqdm import tqdm
    
    """
    maf - maf file from S3Cohort
    cna_gene_file should be generated by get_cna_gene_file()
    fusion_data - raw_fusions from S3Cohort
    """
    
    sample_annotation = pd.DataFrame({
        'Sample.ID': maf.Tumor_Sample_Barcode.sort_values().unique(),
        'Copy.Number': 0,
        'BCL2.transloc': 'NA',
        'BCL6.transloc': 'NA'
    })
    
    if (cna_gene_file is None) & (fusion_data is None):
        print('Sample annotation done')
    else:
        for i in tqdm(sample_annotation['Sample.ID'], desc = '{Generating sample annotation}'):
            if (cna_gene_file is not None):
                if i in cna_gene_file.Sample.values:
                    sample_annotation.loc[sample_annotation['Sample.ID'] == i, 'Copy.Number'] = 1
            if (fusion_data is not None):
                if (i in fusion_data.loc[(fusion_data['Gene B'] == 'BCL6') & (fusion_data['Gene A'] == 'IGH@'), 'Sample'].drop_duplicates().values):
                    sample_annotation.loc[sample_annotation['Sample.ID'] == i, 'BCL6.transloc'] = 1
                if (i in fusion_data.loc[(fusion_data['Gene B'] == 'BCL2') & (fusion_data['Gene A'] == 'IGH@'), 'Sample'].drop_duplicates().values):
                    sample_annotation.loc[sample_annotation['Sample.ID'] == i, 'BCL2.transloc'] = 1
        print('Sample annotation done')

    return sample_annotation

In [19]:
def predict_genclasses(maf, cna_nt = None, segments = None, fusion_data = None):
    
    
    """Performs a prediction of Genclasses from maf, cna_genes, cna_segments and fusion files.
    Maf, cna_nt, segments and fusions should be obtained from S3Cohort. 
    For cna_nt genes should be in raws, samples - in columns
    Returns a dictionary with  Prediction, Compare, model.9, Fullout, Warn.set, modset. 
    Prediction key contains predicted genclasses. Compare contains statistics"""
    
    import rpy2.robjects as ro
    from rpy2.robjects import pandas2ri
    
    #Files for prefiction
    
    print('Data processing for prediction')
    
    Mutation_flat = process_mutation_file(maf)
    
    if 'TRUNC' in Mutation_flat.Type.unique():
        trunc = True
    else:
        trunc = False
    
    mut_genelist = get_mutation_genlist(Mutation_flat)
    
    if (cna_nt is None):
        Flatfile = Mutation_flat
        print('Flatfile done')
        CGH = 1
        Copy_flat = 'NULL'
        CGH_genelist = 'NULL'
        Sample_annot = sample_annotation(maf, cna_gene_file=None, fusion_data=fusion_data)
        Sample_annot = Sample_annot.astype(str)
        Sample_annot['Copy.Number'] = Sample_annot['Copy.Number'].astype(int)
    else:   
        Copy_flat = get_cna_gene_file(cna_nt)
        Flatfile = Copy_flat.append(Mutation_flat)
        print('Flatfile done')
        CGH = 0
        CGH_genelist = get_unique_cna_genes(Copy_flat)
        Sample_annot = sample_annotation(maf, cna_gene_file=Copy_flat, fusion_data=fusion_data)
        Sample_annot = Sample_annot.astype(str)
        Sample_annot['Copy.Number'] = Sample_annot['Copy.Number'].astype(int)
    
    Flatfile = Flatfile.astype(str)
    
    if (segments is None) | (CGH == 1):
        Arm_file = 'NULL'
    else:
        segments_arm = get_segments_arm(segments)
        Arm_file = get_arm_file(segments_arm)
        Arm_file = Arm_file.astype(str)
    

    pandas2ri.activate()
    
    run_predict_genclasses = ro.r(
    """
        function(Mutation_flat, Copy_flat, Flatfile, Sample_annot,  mut_genelist, CGH_genelist, Arm_file, CGH, trunc){
    
            source('/uftp/projects/GenClass_Prediction/Model9_original.R')

            options(stringsAsFactors = FALSE)
            Warnset=NULL

            #flags that should be set before running
            CGH.class=CGH 
            Test.set=c("BN2","EZB","MCD","N1","ST2","A53")
            HasL265P=F
            Has.Trunc=trunc
            
            print('Static data')
            
            load('/uftp/projects/GenClass_Prediction/Model_weights.RData', .GlobalEnv)
            
            mut_genelist=mut_genelist[,1]
            
            if (Arm_file == 'NULL'){
                Arm_file = NULL
            }
            
            
            if (Copy_flat == 'NULL'){
                Copy_flat = NULL
            }

            if (CGH.class == 1){
                Arm_file = NULL
                CGH_genelist=unique(Full.Uber.2019[,3])
            }
            else{
                CGH_genelist = CGH_genelist[,1]
            }
            
            
            Sample_annot$BCL2.transloc = as.numeric(Sample_annot$BCL2.transloc)
            Sample_annot$BCL6.transloc = as.numeric(Sample_annot$BCL6.transloc)
            
            print('Starting prediction')
            
            result=Predict9(
                Flatfile,
                Sample_annot,
                Index.Flat,
                Genclass.lab,
                CGH.class=CGH.class,
                Full.Uber.2019,
                Fullmat.2019,
                originalpred=originalpred,
                Has.CGH=Study.samp[,2]==1,
                Arm.file=Arm_file,
                CGH.genelist=CGH_genelist,
                mut.genelist=mut_genelist,
                CalcWilcox=F,
                Test.set=Test.set,
                Has.Trunc=Has.Trunc)
            
            
            return(result)
        
        }
    """
    )
    
    predict = run_predict_genclasses(Mutation_flat, Copy_flat, Flatfile, Sample_annot,  
                                    mut_genelist, CGH_genelist, Arm_file, CGH, trunc)
    
    result = {}
    
    result['Prediction'] = predict[0]
    result['Compare'] = predict[1]
    result['model.9'] = predict[2]
    result['Fullout'] = predict[3]
    result['Warn.set'] = predict[4]
    result['modset'] = predict[5]
    
    return result

# Test

In [11]:
import pandas as pd
from bioreactor.cohort_data import S3Cohort

In [12]:
pats_with_norms = ['1002-A',
 '1004-A',
 '1014-A',
 '1015-A',
 '1017-A',
 '1019-A',
 '1020-A',
 '1021-A',
 '1023-A',
 '1024-A',
 '1028-A',
 '1033-A',
 '1034-A',
 '1035-A',
 '1037-A',
 '1039-A',
 '1041-A',
 '1043-A',
 '1046-A',
 '1049-A',
 '1053-A',
 '1054-A',
 '1058-A',
 '1020-B',
 '1032-B',
 '1049-B']

cohort = S3Cohort('MSKCC_DLBCL_Batlevi_CART', patients=pats_with_norms)

In [ ]:
g_maf = cohort.maf

In [ ]:
g_cna_nt = cohort.cna_nt

In [ ]:
g_fusions = cohort.raw_fusions

In [23]:
g_segments = cohort.segments

In [25]:
genclass = predict_genclasses(maf = g_maf, cna_nt=g_cna_nt.T, segments=g_segments, fusion_data=g_fusions)

Data processing for prediction
Generating Mutation file


{Generating cna_gene_file}: 100%|██████████| 26/26 [00:00<00:00, 811.44it/s]

Mutation file done



{Generating sample annotation}: 100%|██████████| 26/26 [00:00<00:00, 224.26it/s]

Flatfile done
Sample annotation done



{Generating arm_file}: 100%|██████████| 26/26 [00:00<00:00, 3117.10it/s]


[1] "Static data"
[1] "Starting prediction"
Run 1 
1 1 1 3619 1 4169 8.861111e-36 
2 2 2 6075 10 4159 1.185831e-27 
3 3 3 27695 7 4152 6.47551e-27 
4 16 4 43068 12 4140 3.386676e-15 
5 17 5 11213 12 4128 1.529684e-14 
6 20 6 42021|42019 19 4098 3.871459e-14 
7 23 7 51733 13 4085 7.316028e-14 
8 36 8 70385 8 4077 8.380594e-13 
9 65 8 70385 3 4074 1.973819e-09 
10 72 10 111593 6 4068 6.550313e-07 
11 76 11 12884 18 4050 2.140334e-06 
12 84 11 12884 6 4044 5.285493e-06 
13 90 11 12884 6 4038 1.311737e-05 
14 95 14 97191 13 4018 3.073526e-05 
15 97 15 17477 12 4006 3.475619e-05 
16 100 15 17477 3 4003 3.738354e-05 
17 107 17 62294 13 3990 4.738346e-05 
18 119 17 62294 5 3985 7.892635e-05 
19 128 17 62294 2 3983 0.0001604264 
20 130 20 45714 12 3971 0.0001735275 
21 147 20 45714 4 3967 0.0003541207 
22 170 20 45714 3 3964 0.0007568589 
23 171 20 45714 3 3961 0.0007568589 
24 176 20 45714 1 3960 0.0008802674 
Run 2 
1 1 1 3559 12 5433 2.98165e-36 
2 7 2 50511|50504 23 5234 4.549503e-28 
3 11

In [26]:
genclass['Prediction']

,Sample.Name,Copy.Number,BCL2.Translocation,BCL6.Translocation,Model.Used,Confidence.BN2,Confidence.EZB,Confidence.MCD,Confidence.N1,Confidence.ST2,Confidence.A53,BN2.Feature.Count,EZB.Feature.Count,MCD.Feature.Count,N1.Feature.Count,ST2.Feature.Count,A53.Feature.Count,Subtype.Prediction
0,1002-A,Available,Not Available,Not Available,NoFus,0.07,0.99,0.01,0.08,0.99,0.99,1.0,7.0,3.0,0.0,6.0,10.0,EZB/ST2/A53
1,1004-A,Available,Not Available,Not Available,NoFus,0.09,0.99,0.00,0.08,0.38,0.54,1.0,7.0,2.0,0.0,3.0,4.0,EZB
2,1014-A,Available,Not Available,Not Available,NoFus,0.16,0.31,0.00,0.08,0.23,0.96,1.0,2.0,2.0,0.0,3.0,5.0,A53
3,1015-A,Available,Not Available,Not Available,NoFus,0.03,0.05,0.49,0.08,0.00,0.04,0.0,2.0,5.0,0.0,1.0,2.0,Other
4,1017-A,Available,Not Available,Not Available,NoFus,0.46,0.13,0.99,0.08,0.98,0.32,1.0,2.0,8.0,0.0,7.0,3.0,MCD/ST2
5,1019-A,Available,Not Available,Not Available,NoFus,0.79,0.16,0.40,0.08,0.16,0.99,3.0,2.0,4.0,0.0,3.0,9.0,A53
6,1020-A,Available,Not Available,Not Available,NoFus,0.73,0.20,0.99,0.99,0.58,0.03,4.0,2.0,9.0,1.0,4.0,2.0,MCD/N1
7,1020-B,Available,Not Available,yes,NoBCL2Fus,0.93,0.06,0.89,0.08,0.06,0.00,5.0,1.0,6.0,0.0,2.0,0.0,BN2
8,1021-A,Available,Not Available,Not Available,NoFus,0.03,0.98,0.02,0.08,0.63,0.00,0.0,4.0,3.0,0.0,4.0,1.0,EZB
9,1023-A,Available,Not Available,Not Available,NoFus,0.22,0.90,0.04,0.08,0.91,0.44,2.0,3.0,3.0,0.0,5.0,4.0,EZB/ST2
